# **I. Model Selection & Baseline Leaderboard**

## **I.1. Objective & Experiment Design**

The objective of this phase is to establish a **performance baseline** using default algorithms. This "Leaderboard" serves two critical functions:

1.  **Complexity Justification:** It determines whether complex non-linear models (like XGBoost) provide a statistically significant advantage over simpler linear models (Logistic Regression).
2.  **Overfitting Detection:** By simultaneously measuring performance on the **Training Set** and the **Validation Set**, we identify models that "memorize" the data rather than generalizing the underlying cluster logic.

**The Candidate Algorithms:**

*   **Logistic Regression:** A linear baseline. If this performs well, the cluster boundaries are simple and linear.
*   **Random Forest:** A bagging ensemble. It captures non-linear interactions and serves as the direct comparator to Group 1's approach.
*   **XGBoost / LightGBM:** Gradient boosting machines. These are the current industry standard for tabular data, expected to handle the "Augmented Features" (like delivery delays) most effectively.

## **I.2. Environment Setup & Metric Definition**
To ensure rigorous evaluation, we define a standardized **Evaluation Function** that will be applied identically to all models.

**Key Metrics Selection:**
*   **F1-Score (Weighted):** The primary success metric. It accounts for the **Class Imbalance** (Cluster 2 is only 3%) by weighting the score of each class by its support (number of true instances).
*   **Log Loss (Cross-Entropy):** Measures the **confidence** of predictions. A model that predicts the correct class with 51% probability is "worse" than one predicting with 90% probability, even if accuracy is identical.
*   **ROC-AUC (One-vs-Rest):** Measures the model's ability to distinguish between classes across different probability thresholds.


**Code Implementation: Setup & Metrics**

**Purpose:** Import necessary libraries, load the stratified datasets created in ***Data Engineering*** section, and define a reusable function to calculate and display the required metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn Models & Metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import f1_score, log_loss, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

# Gradient Boosting Libraries
import xgboost as xgb
import lightgbm as lgb

In [2]:
# --- CONFIGURATION ---
DATA_DIR = "data/"

def load_stratified_data():
    """
    Loads the 6 split files generated in Task 3.
    Returns X_train, y_train, X_val, y_val, X_test, y_test
    """
    print("--- [START] Loading Stratified Data ---")
    try:
        X_train = pd.read_csv(f"{DATA_DIR}X_train.csv")
        y_train = pd.read_csv(f"{DATA_DIR}y_train.csv").squeeze() # squeeze to convert DF to Series
        X_val = pd.read_csv(f"{DATA_DIR}X_val.csv")
        y_val = pd.read_csv(f"{DATA_DIR}y_val.csv").squeeze()
        X_test = pd.read_csv(f"{DATA_DIR}X_test.csv")
        y_test = pd.read_csv(f"{DATA_DIR}y_test.csv").squeeze()
        
        print(f"    Train Shape: {X_train.shape}")
        print(f"    Val Shape:   {X_val.shape}")
        print(f"    Test Shape:  {X_test.shape}")
        print("--- [END] Data Loaded Successfully ---\n")
        return X_train, y_train, X_val, y_val, X_test, y_test
    except FileNotFoundError:
        print("    [ERROR] Split files not found. Please run Task 3 first.")
        return None, None, None, None, None, None

def evaluate_model(model, X, y, dataset_name="Validation"):
    """
    Calculates Macro F1 (fairness), Weighted F1, and Log Loss.
    Prints a detailed per-class breakdown.
    """
    # 1. Generate Predictions
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)
    
    # 2. Calculate Metrics
    # Macro: Treats all classes equally (Crucial for Cluster 2)
    f1_macro = f1_score(y, y_pred, average='macro')
    # Weighted: Account for population size
    f1_weighted = f1_score(y, y_pred, average='weighted')
    
    # Log Loss
    try:
        ll = log_loss(y, y_prob)
    except ValueError:
        ll = np.nan
        
    # ROC-AUC
    try:
        auc = roc_auc_score(y, y_prob, multi_class='ovr', average='weighted')
    except ValueError:
        auc = np.nan

    # 3. Print Results
    print(f"[{dataset_name} Performance]")
    print(f"    F1-Score (Macro):    {f1_macro:.4f}  <-- EQUAL WEIGHT (Fairness)")
    print(f"    F1-Score (Weighted): {f1_weighted:.4f}")
    print(f"    Log Loss:            {ll:.4f}")
    
    # 4. Print Per-Class Report (Only for Validation to avoid clutter)
    if dataset_name == "Validation":
        print("\n    >>> Per-Class Performance (Check Cluster 2!):")
        print(classification_report(y, y_pred))
    
    return {
        "F1_Macro": f1_macro,
        "F1_Weighted": f1_weighted,
        "ROC_AUC": auc,
        "Log_Loss": ll
    }

# Execute Loading
X_train, y_train, X_val, y_val, X_test, y_test = load_stratified_data()

--- [START] Loading Stratified Data ---
    Train Shape: (65350, 10)
    Val Shape:   (14004, 10)
    Test Shape:  (14004, 10)
--- [END] Data Loaded Successfully ---



## **I.3. Baseline Models: Logistic Regression & Random Forest**

*   **Logistic Regression:**
    *   **Role:** The "Simplicity Test."
    *   **Hypothesis:** If the clusters created by K-Means are geometrically distinct (e.g., separated by clear hyperplanes), this simple linear model should perform surprisingly well. If it fails (low F1 score), it confirms the relationships between inputs (like `credit_card_usage`) and clusters are **non-linear**.
    *   **Configuration:** We use `max_iter=1000` to ensure the solver converges, as our dataset has 93k rows.

*   **Random Forest Classifier:**
    *   **Role:** The "Direct Comparator."
    *   **Hypothesis:** This mimics Group 1's approach but applied to our **stratified** data. Random Forest handles non-linearity well but is prone to **overfitting** (high Train score, low Val score) if trees are allowed to grow too deep.
    *   **Configuration:** We use a `random_state` seed for reproducibility.

**Code Implementation: Training LR and RF**

**Purpose:** Train both models on `X_train`, then evaluate them on **both** `X_train` (to check for memorization) and `X_val` (to check for generalization).

In [3]:
# %%
# Store results for the Leaderboard comparison later
leaderboard = []

# --- 1. LOGISTIC REGRESSION (Linear Baseline) ---
print("\n=== Model 1: Logistic Regression ===")
lr_model = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=3000, random_state=42)
lr_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_lr = evaluate_model(lr_model, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_lr = evaluate_model(lr_model, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "Logistic Regression",
    "Train_F1_Macro": train_metrics_lr["F1_Macro"],
    "Val_F1_Macro": val_metrics_lr["F1_Macro"],
    "Val_LogLoss": val_metrics_lr["Log_Loss"],
})

# --- 2. RANDOM FOREST (Bagging Ensemble) ---
print("\n=== Model 2: Random Forest (Default) ===")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_rf = evaluate_model(rf_model, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_rf = evaluate_model(rf_model, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "Random Forest",
    "Train_F1_Macro": train_metrics_rf["F1_Macro"],
    "Val_F1_Macro": val_metrics_rf["F1_Macro"],
    "Val_LogLoss": val_metrics_rf["Log_Loss"],
})



=== Model 1: Logistic Regression ===


d:\_Python\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
d:\_Python\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 3000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=3000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Macro):    0.9676  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9505
    Log Loss:            0.1144
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Macro):    0.9650  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9506
    Log Loss:            0.1184

    >>> Per-Class Performance (Check Cluster 2!):
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      6866
           1       0.94      0.93      0.94      5401
           2       0.99      0.97      0.98       420
           3       0.99      0.99      0.99      1317

    accuracy                           0.95     14004
   macro avg       0.97      0.96      0.96     14004
weighted avg       0.95      0.95      0.95     14004


=== Model 2: Random Forest (Default) ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Macro):    1.0000  <-- EQUAL WEIGHT (Fa

## **I.4. Gradient Boosting Challengers: XGBoost & LightGBM**

To complete the baseline assessment, we deploy two industry-standard Gradient Boosting machines. Unlike Random Forest (which builds trees in parallel), these models build trees **sequentially**, with each new tree correcting the errors of the previous one.

*   **XGBoost (eXtreme Gradient Boosting):**
    *   **Strength:** Known for precision and regularization (preventing overfitting).
    *   **Hypothesis:** It should provide the best **Log Loss** (probability calibration) of all models. XGBoost should match Random Forest's accuracy (99%+) but potentially with lower **Log Loss** (better probability confidence).
*   **LightGBM (Light Gradient Boosting Machine):**
    *   **Strength:** Optimized for speed and efficiency using histogram-based learning.
    *   **Hypothesis:** It serves as a "Speed Test." If it matches XGBoost's accuracy but trains 5x faster, it becomes the preferred candidate for deployment.


In [4]:
# --- 3. XGBOOST (Gradient Boosting) ---
print("\n=== Model 3: XGBoost Classifier ===")
xgb_model = xgb.XGBClassifier(objective="multi:softprob", num_class=4, n_estimators=100, random_state=42, n_jobs=-1)
xgb_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_xgb = evaluate_model(xgb_model, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_xgb = evaluate_model(xgb_model, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "XGBoost",
    "Train_F1_Macro": train_metrics_xgb["F1_Macro"],
    "Val_F1_Macro": val_metrics_xgb["F1_Macro"],
    "Val_LogLoss": val_metrics_xgb["Log_Loss"],
})

# --- 4. LIGHTGBM (High-Efficiency Boosting) ---
print("\n=== Model 4: LightGBM Classifier ===")
lgb_model = lgb.LGBMClassifier(
    objective="multiclass", num_class=4, n_estimators=100, random_state=42, n_jobs=-1, verbosity=-1
)
lgb_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_lgb = evaluate_model(lgb_model, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_lgb = evaluate_model(lgb_model, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "LightGBM",
    "Train_F1_Macro": train_metrics_lgb["F1_Macro"],
    "Val_F1_Macro": val_metrics_lgb["F1_Macro"],
    "Val_LogLoss": val_metrics_lgb["Log_Loss"],
})



=== Model 3: XGBoost Classifier ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Macro):    1.0000  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 1.0000
    Log Loss:            0.0021
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Macro):    0.9979  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9964
    Log Loss:            0.0092

    >>> Per-Class Performance (Check Cluster 2!):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6866
           1       1.00      0.99      1.00      5401
           2       1.00      1.00      1.00       420
           3       1.00      1.00      1.00      1317

    accuracy                           1.00     14004
   macro avg       1.00      1.00      1.00     14004
weighted avg       1.00      1.00      1.00     14004


=== Model 4: LightGBM Classifier ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Macro)

## **I.5. "Modern" Tree Alternatives: CatBoost & ExtraTrees**

*  **CatBoost:** The third giant of Gradient Boosting (alongside XGBoost and LightGBM). It handles categorical features differently and often generalizes better.

*  **ExtraTrees (Extremely Randomized Trees):** A variant of Random Forest that introduces more randomness. It is often faster and less prone to overfitting than standard RF.

In [5]:
# --- 5. CATBOOST (Robust Boosting) ---
print("\n=== Model 5: CatBoost Classifier ===")
cb_model = CatBoostClassifier(
    loss_function="MultiClass",
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=0,
    allow_writing_files=False,
)
cb_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_cb = evaluate_model(cb_model, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_cb = evaluate_model(cb_model, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "CatBoost",
    "Train_F1_Macro": train_metrics_cb["F1_Macro"],
    "Val_F1_Macro": val_metrics_cb["F1_Macro"],
    "Val_LogLoss": val_metrics_cb["Log_Loss"],
})

# --- 6. EXTRA TREES (Randomized Ensemble) ---
print("\n=== Model 6: Extra Trees Classifier ===")
et_model = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1)
et_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_et = evaluate_model(et_model, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_et = evaluate_model(et_model, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "Extra Trees",
    "Train_F1_Macro": train_metrics_et["F1_Macro"],
    "Val_F1_Macro": val_metrics_et["F1_Macro"],
    "Val_LogLoss": val_metrics_et["Log_Loss"],
})



=== Model 5: CatBoost Classifier ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Macro):    0.9994  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9990
    Log Loss:            0.0112
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Macro):    0.9985  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9974
    Log Loss:            0.0134

    >>> Per-Class Performance (Check Cluster 2!):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6866
           1       1.00      1.00      1.00      5401
           2       1.00      1.00      1.00       420
           3       1.00      1.00      1.00      1317

    accuracy                           1.00     14004
   macro avg       1.00      1.00      1.00     14004
weighted avg       1.00      1.00      1.00     14004


=== Model 6: Extra Trees Classifier ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Ma

## **I.6. Non-Tree Alternatives: KNN & Neural Net**

*  **K-Nearest Neighbors (KNN):** A distance-based algorithm. Since our clusters were generated by K-Means (distance-based), KNN should theoretically perform very well.
    *   *Note:* KNN requires feature scaling, so we will use a `Pipeline` with `StandardScaler`.

*  **MLP Classifier (Neural Network):** A simple Deep Learning baseline. This checks if non-linear feature interactions are better captured by neurons than trees.

**Crucial Note:** Unlike trees, KNN and Neural Networks (MLP) are sensitive to the scale of data (e.g., Monetary=500 vs Frequency=1). We **must** scale the data using `StandardScaler` inside a Pipeline for these models to work correctly.

In [6]:
# --- 7. K-NEAREST NEIGHBORS (Distance Based) ---
print("\n=== Model 7: K-Nearest Neighbors (KNN) ===")
knn_pipeline = Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=10, n_jobs=-1))])
knn_pipeline.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_knn = evaluate_model(knn_pipeline, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_knn = evaluate_model(knn_pipeline, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "KNN",
    "Train_F1_Macro": train_metrics_knn["F1_Macro"],
    "Val_F1_Macro": val_metrics_knn["F1_Macro"],
    "Val_LogLoss": val_metrics_knn["Log_Loss"],
})

# --- 8. MLP CLASSIFIER (Neural Network) ---
print("\n=== Model 8: MLP (Neural Network) ===")
mlp_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)),
])
mlp_pipeline.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_mlp = evaluate_model(mlp_pipeline, X_train, y_train, dataset_name="Train")
print("  > Evaluating on VALIDATION set...")
val_metrics_mlp = evaluate_model(mlp_pipeline, X_val, y_val, dataset_name="Validation")

# [FIX] Updated Keys to match Macro F1
leaderboard.append({
    "Model": "Neural Network (MLP)",
    "Train_F1_Macro": train_metrics_mlp["F1_Macro"],
    "Val_F1_Macro": val_metrics_mlp["F1_Macro"],
    "Val_LogLoss": val_metrics_mlp["Log_Loss"],
})



=== Model 7: K-Nearest Neighbors (KNN) ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Macro):    0.9798  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9685
    Log Loss:            0.0789
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Macro):    0.9734  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9580
    Log Loss:            0.1260

    >>> Per-Class Performance (Check Cluster 2!):
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      6866
           1       0.96      0.93      0.95      5401
           2       1.00      0.99      1.00       420
           3       1.00      0.99      0.99      1317

    accuracy                           0.96     14004
   macro avg       0.98      0.97      0.97     14004
weighted avg       0.96      0.96      0.96     14004


=== Model 8: MLP (Neural Network) ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score

## **I.7. Conclusion: Baseline Model Selection**

In [7]:
# --- SAVE & SELECT TOP 3 ---
print("\n=== I.7. Leaderboard & Selection (Macro-Averaged) ===")
leaderboard_path = f"{DATA_DIR}phase_a_leaderboard.csv"
df_leaderboard = pd.DataFrame(leaderboard)

# Calculate Overfitting Gap using MACRO F1
df_leaderboard['Overfitting_Gap'] = df_leaderboard['Train_F1_Macro'] - df_leaderboard['Val_F1_Macro']

# Sort by Validation MACRO F1 (Descending) -> Best balance across all classes
# We use Macro F1 as primary, Log Loss as tie-breaker
df_leaderboard = df_leaderboard.sort_values(by=["Val_F1_Macro", "Val_LogLoss"], ascending=[False, True])

# Save
df_leaderboard.to_csv(leaderboard_path, index=False)
print(f"[SAVED] Full Leaderboard saved to: {leaderboard_path}")

# Display
print(df_leaderboard[['Model', 'Val_F1_Macro', 'Val_LogLoss', 'Overfitting_Gap']].to_markdown(index=False))

# Select Top 3 Models
top_3_models = df_leaderboard['Model'].head(3).tolist()
print(f"\n[DECISION] The Top 3 Candidates for Hyperparameter Tuning are: {top_3_models}")


=== I.7. Leaderboard & Selection (Macro-Averaged) ===
[SAVED] Full Leaderboard saved to: data/phase_a_leaderboard.csv
| Model                |   Val_F1_Macro |   Val_LogLoss |   Overfitting_Gap |
|:---------------------|---------------:|--------------:|------------------:|
| CatBoost             |       0.998511 |    0.013401   |       0.000912699 |
| LightGBM             |       0.998026 |    0.00968221 |       0.0019207   |
| XGBoost              |       0.997891 |    0.00920968 |       0.00210903  |
| Random Forest        |       0.997147 |    0.0229231  |       0.00285284  |
| Neural Network (MLP) |       0.996534 |    0.0135822  |       0.00163022  |
| Extra Trees          |       0.994428 |    0.0568825  |       0.00557246  |
| KNN                  |       0.973361 |    0.12596    |       0.00648751  |
| Logistic Regression  |       0.964983 |    0.118396   |       0.00260455  |

[DECISION] The Top 3 Candidates for Hyperparameter Tuning are: ['CatBoost', 'LightGBM', 'XGBoost']


### **I.7.1. The Model Leaderboard**

We successfully evaluated 8 distinct algorithms ranging from simple linear baselines to complex neural networks and gradient boosting machines. The results are sorted by **Validation Macro F1-Score** (Primary) and **Log Loss** (Secondary).

| Model | Val_F1_Macro | Val_LogLoss | Overfitting_Gap | Rank |
| :--- | :--- | :--- | :--- | :--- |
| **CatBoost** | **0.9985** | 0.0134 | 0.0009 | **1** |
| **LightGBM** | **0.9980** | 0.0097 | 0.0019 | **2** |
| **XGBoost** | **0.9979** | **0.0092** | 0.0021 | **3** |
| Random Forest | 0.9971 | 0.0229 | 0.0029 | 4 |
| Neural Network (MLP) | 0.9965 | 0.0136 | 0.0016 | 5 |
| Extra Trees | 0.9944 | 0.0569 | 0.0056 | 6 |
| KNN | 0.9734 | 0.1260 | 0.0065 | 7 |
| Logistic Regression | 0.9650 | 0.1184 | 0.0026 | 8 |

### **I.7.2. Key Findings & Analysis**

**A. The Dominance of Gradient Boosting**

The "Big Three" boosting algorithms (**CatBoost, LightGBM, XGBoost**) monopolized the top 3 positions. They achieved near-perfect classification scores (>99.7%), effectively solving the problem.

*   **CatBoost** achieved the highest **Macro F1 (0.9985)**, indicating it handles the minority "VIP" cluster slightly better than its competitors.

*   **XGBoost** achieved the lowest **Log Loss (0.0092)**. While it made slightly more classification errors than CatBoost, it was mathematically more **confident** in its probabilities.

**B. Validation of the Surrogate Task**

The exceptionally high scores across all models (even Logistic Regression at 96.5%) confirm a critical theoretical point: **The K-Means clusters are deterministic and well-separated.**

*   Since the Target Variable ($Y$) was generated using the Input Features ($X$), there is a direct mathematical relationship between them.

*   The models successfully "reverse-engineered" the K-Means distance logic. This grants us high confidence that the **Feature Importance** we extract in Task 6 will be an accurate representation of the cluster definitions.

**C. Generalization Capability**

The **Overfitting Gap** (Train Score minus Validation Score) is negligible across the board (~0.001 to 0.005). This proves that the **Stratified Sampling** strategy worked perfectly; the Validation set is a true mirror of the Training set, and the models are capturing genuine patterns rather than memorizing noise.

### **I.7.3. Selection Decision**

To satisfy the requirement of "Checking Overfitting" and "Tuning Various Models," we select the top 3 performers for Phase B (Hyperparameter Optimization).

1.  **CatBoost:** The Accuracy Leader.
2.  **LightGBM:** The Speed/Efficiency Leader.
3.  **XGBoost:** The Probability/Confidence Leader (Lowest Log Loss).

# **II. Hyperparameter Tuning (Top 3 Candidates)**

## **II.1. Tuning Strategy: Bayesian Optimization via Optuna**

To extract the maximum performance from our top 3 candidates (**CatBoost, LightGBM, XGBoost**), we abandon brute-force methods (Grid Search) in favor of **Tree-structured Parzen Estimator (TPE)**, implemented via the **Optuna** framework.

### **II.1.1. Methodology Selection**

*   **Why not Grid Search?** Grid search suffers from the "Curse of Dimensionality." Tuning 6 parameters with 5 options each results in $5^6 = 15,625$ trials per model. This is computationally infeasible.

*   **Why not Hyperband?** Hyperband relies on aggressive early stopping (killing trials that perform poorly in early iterations). For Gradient Boosting, this is risky; models with low learning rates often start slow but achieve the best final convergence. Killing them early leads to sub-optimal results.

*   **Why Bayesian Optimization (TPE)?** TPE is an "informed" search methods. It builds a probabilistic model ($P(Score | Hyperparameters)$) based on past trials to predict which hyperparameters are likely to yield better results. It balances **Exploration** (trying new values) and **Exploitation** (refining the best values).

### **II.1.2. The "Beat the Default" Heuristic**

A common pitfall in tuning is producing a model *worse* than the default settings (Negative Lift). This happens when the search space is too restrictive or the optimizer gets stuck in a local minimum.

*   **The Solution:** We utilize Optuna's `enqueue_trial` feature.

*   **Mechanism:** We explicitly force the optimizer to evaluate the **Default Hyperparameters** as Trial 0.

*   **Guarantee:** This ensures the final "Best Params" are mathematically guaranteed to be **$\ge$ Default Performance**.

### **II.1.3. Optimization Target**

*   **Metric:** **Log Loss (Cross-Entropy)**.

*   **Reasoning:** F1-Score is a "step function" (predictions are either 0 or 1). Log Loss is continuous and smooth, providing the optimizer with better gradients to find the global minimum. A model with lower Log Loss is more confident and robust.


## **II.2. Tuning Models**

### **II.2.1. Model 1 (CatBoost)**

*   **Specific Constraint:** CatBoost is unique because it is effectively "self-tuning." Its default parameters are extremely strong. To beat them, we must tune the `depth`, `l2_leaf_reg`, and `random_strength` without breaking its internal symmetries.
*   **Trials:** We will run **30 trials**, which is sufficient for CatBoost's smaller hyperparameter space.

In [ ]:
# %%
# #########################################################
# ## Model 1 Tuning: CatBoost
# #########################################################

import optuna
import json
import os
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss

# --- CONFIGURATION ---
CB_PARAMS_FILE = "data/best_catboost_params.json"

def objective_catboost(trial):
    params = {
        'loss_function': 'MultiClass',
        'iterations': 500, # Fixed budget
        'verbose': 0,
        'random_seed': 42,
        'allow_writing_files': False,
        
        # Search Space
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train)
    
    # Optimize for Log Loss (Confidence)
    y_prob = model.predict_proba(X_val)
    return log_loss(y_val, y_prob)

# --- EXECUTION LOGIC ---
if os.path.exists(CB_PARAMS_FILE):
    print(f"--- [INFO] Found saved parameters in {CB_PARAMS_FILE}. Skipping optimization. ---")
    with open(CB_PARAMS_FILE, 'r') as f:
        best_params_cb = json.load(f)
else:
    print("--- [START] Tuning CatBoost with Optuna (30 Trials) ---")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study_cb = optuna.create_study(direction='minimize')
    
    # 1. Enqueue Default Params (The Baseline)
    # This guarantees we don't get a result worse than default
    study_cb.enqueue_trial({
        'depth': 6,
        'learning_rate': 0.1,
        'l2_leaf_reg': 3.0,
        'random_strength': 1.0,
        'bagging_temperature': 1.0,
        'border_count': 254
    })
    
    # 2. Run Optimization
    study_cb.optimize(objective_catboost, n_trials=30)
    
    best_params_cb = study_cb.best_params
    
    # Save to JSON
    with open(CB_PARAMS_FILE, 'w') as f:
        json.dump(best_params_cb, f, indent=4)
    print(f"--- [SAVED] Best parameters saved to {CB_PARAMS_FILE} ---")

print("\n--- [RESULT] Best CatBoost Params ---")
for key, value in best_params_cb.items():
    print(f"    {key}: {value}")

d:\_Python\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- [START] Tuning CatBoost with Optuna (30 Trials) ---
--- [SAVED] Best parameters saved to data/best_catboost_params.json ---

--- [RESULT] Best CatBoost Params ---
    depth: 5
    learning_rate: 0.29731176900667866
    l2_leaf_reg: 7.131875916404689e-08
    random_strength: 4.953331414034946e-06
    bagging_temperature: 0.204759048712155
    border_count: 190


### **II.2.2. Model 2 (LightGBM)**

LightGBM is highly sensitive to `num_leaves` and `min_child_samples`. If we don't constrain these relative to the dataset size, it overfits easily.

*   **Strategy:** We constrain `num_leaves` to be less than $2^{\text{max\_depth}}$ to prevent tree explosion.

*   **Search Space:** Focus on `num_leaves`, `max_depth`, and `feature_fraction`.


In [ ]:
# #########################################################
# ## Model 2 Tuning: LightGBM
# #########################################################

# --- CONFIGURATION ---
LGB_PARAMS_FILE = "data/best_lgbm_params.json"

def objective_lgbm(trial):
    params = {
        'objective': 'multiclass',
        'num_class': 4,
        'n_estimators': 500,
        'verbosity': -1,
        'random_state': 42,
        'n_jobs': -1,
        
        # Search Space
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    
    y_prob = model.predict_proba(X_val)
    return log_loss(y_val, y_prob)

# --- EXECUTION LOGIC ---
if os.path.exists(LGB_PARAMS_FILE):
    print(f"--- [INFO] Found saved parameters in {LGB_PARAMS_FILE}. Skipping optimization. ---")
    with open(LGB_PARAMS_FILE, 'r') as f:
        best_params_lgbm = json.load(f)
else:
    print("--- [START] Tuning LightGBM with Optuna (30 Trials) ---")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study_lgb = optuna.create_study(direction='minimize')
    
    # Enqueue Defaults
    study_lgb.enqueue_trial({
        'learning_rate': 0.1,
        'num_leaves': 31,
        'max_depth': -1, # Default is unconstrained, we map -1 to 12 roughly in space
        'min_child_samples': 20,
        'subsample': 1.0,
        'colsample_bytree': 1.0,
        'reg_alpha': 1e-8,
        'reg_lambda': 1e-8
    })
    
    study_lgb.optimize(objective_lgbm, n_trials=30)
    best_params_lgbm = study_lgb.best_params
    
    with open(LGB_PARAMS_FILE, 'w') as f:
        json.dump(best_params_lgbm, f, indent=4)
    print(f"--- [SAVED] Best parameters saved to {LGB_PARAMS_FILE} ---")

print("\n--- [RESULT] Best LightGBM Params ---")
for key, value in best_params_lgbm.items():
    print(f"    {key}: {value}")

--- [START] Tuning LightGBM with Optuna (30 Trials) ---


d:\_Python\.venv\Lib\site-packages\optuna\trial\_trial.py:656: UserWarning: Fixed parameter 'max_depth' with value -1 is out of range for distribution IntDistribution(high=12, log=False, low=5, step=1).
  warnings.warn(


--- [SAVED] Best parameters saved to data/best_lgbm_params.json ---

--- [RESULT] Best LightGBM Params ---
    learning_rate: 0.028414792283929843
    num_leaves: 62
    max_depth: 10
    min_child_samples: 46
    subsample: 0.645209418114804
    colsample_bytree: 0.8832829051166367
    reg_alpha: 4.525503194174542e-07
    reg_lambda: 7.290849490307852e-05


### **II.2.3. Model 3 (XGBoost)**

XGBoost is the most mature framework. We reuse the strategy we defined earlier but apply the "Beat the Default" heuristic formally here.

*   **Strategy:** Aggressive regularization (`gamma`, `reg_lambda`) to maintain the "Confidence" advantage it showed in Section I (lowest Log Loss).

*   **Search Space:** Deep trees (`max_depth` up to 10) vs. High Regularization.


In [10]:
# #########################################################
# ## Model 3 Tuning: XGBoost
# #########################################################

# --- CONFIGURATION ---
XGB_PARAMS_FILE = "data/best_xgb_params_v3.json"

def objective_xgb(trial):
    params = {
        'objective': 'multi:softprob',
        'num_class': 4,
        'n_estimators': 500,
        'verbosity': 0,
        'random_state': 42,
        'n_jobs': -1,
        
        # Search Space
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }
    
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    
    y_prob = model.predict_proba(X_val)
    return log_loss(y_val, y_prob)

# --- EXECUTION LOGIC ---
if os.path.exists(XGB_PARAMS_FILE):
    print(f"--- [INFO] Found saved parameters in {XGB_PARAMS_FILE}. Skipping optimization. ---")
    with open(XGB_PARAMS_FILE, 'r') as f:
        best_params_xgb = json.load(f)
else:
    print("--- [START] Tuning XGBoost with Optuna (30 Trials) ---")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study_xgb = optuna.create_study(direction='minimize')
    
    # Enqueue Defaults
    study_xgb.enqueue_trial({
        'learning_rate': 0.3,
        'max_depth': 6,
        'min_child_weight': 1,
        'gamma': 1e-8,
        'subsample': 1.0,
        'colsample_bytree': 1.0,
        'reg_alpha': 1e-8,
        'reg_lambda': 1.0
    })
    
    study_xgb.optimize(objective_xgb, n_trials=30)
    best_params_xgb = study_xgb.best_params
    
    with open(XGB_PARAMS_FILE, 'w') as f:
        json.dump(best_params_xgb, f, indent=4)
    print(f"--- [SAVED] Best parameters saved to {XGB_PARAMS_FILE} ---")

print("\n--- [RESULT] Best XGBoost Params ---")
for key, value in best_params_xgb.items():
    print(f"    {key}: {value}")

--- [START] Tuning XGBoost with Optuna (30 Trials) ---
--- [SAVED] Best parameters saved to data/best_xgb_params_v3.json ---

--- [RESULT] Best XGBoost Params ---
    learning_rate: 0.05821055555916829
    max_depth: 8
    min_child_weight: 1
    gamma: 1.2571402698249565e-08
    subsample: 0.6908553863978453
    colsample_bytree: 0.983093003148984
    reg_alpha: 2.060094700440822e-08
    reg_lambda: 0.05141737398228071


## **II.3. Final Evaluation & Tuned Leaderboard**

In [12]:
# #########################################################
# ## Final Evaluation & Tuned Leaderboard
# #########################################################

print("\n=== Final Evaluation: Default vs. Tuned ===")

# 1. Load Best Parameters
with open("data/best_catboost_params.json", 'r') as f:
    params_cb = json.load(f)
with open("data/best_lgbm_params.json", 'r') as f:
    params_lgbm = json.load(f)
with open("data/best_xgb_params_v3.json", 'r') as f:
    params_xgb = json.load(f)

# 2. Re-Train Models with Best Params
# We need to add back the static parameters (objective, n_jobs, etc.)
# because Optuna only saved the hyperparameters.

# --- Tuned CatBoost ---
final_cb = CatBoostClassifier(
    loss_function='MultiClass', iterations=500, verbose=0, 
    random_seed=42, allow_writing_files=False,
    **params_cb
)
final_cb.fit(X_train, y_train)
metrics_cb = evaluate_model(final_cb, X_val, y_val, dataset_name="Tuned CatBoost")

# --- Tuned LightGBM ---
final_lgbm = lgb.LGBMClassifier(
    objective='multiclass', num_class=4, n_estimators=500, 
    verbosity=-1, random_state=42, n_jobs=-1,
    **params_lgbm
)
final_lgbm.fit(X_train, y_train)
metrics_lgbm = evaluate_model(final_lgbm, X_val, y_val, dataset_name="Tuned LightGBM")

# --- Tuned XGBoost ---
final_xgb = xgb.XGBClassifier(
    objective='multi:softprob', num_class=4, n_estimators=500, 
    verbosity=0, random_state=42, n_jobs=-1,
    **params_xgb
)
final_xgb.fit(X_train, y_train)
metrics_xgb = evaluate_model(final_xgb, X_val, y_val, dataset_name="Tuned XGBoost")

# 3. Create Comparison Table
# Load Baseline (Phase A)
try:
    df_base = pd.read_csv("data/phase_a_leaderboard.csv")
    # Filter for our Top 3
    df_base = df_base[df_base['Model'].isin(['CatBoost', 'LightGBM', 'XGBoost'])].copy()
    df_base = df_base[['Model', 'Val_F1_Macro', 'Val_LogLoss']]
    df_base.columns = ['Model', 'Default_F1', 'Default_LogLoss']
except FileNotFoundError:
    print("[WARNING] phase_a_leaderboard.csv not found. Comparison will be limited.")
    df_base = pd.DataFrame(columns=['Model', 'Default_F1', 'Default_LogLoss'])

# Create Tuned Results DataFrame
tuned_results = [
    {'Model': 'CatBoost', 'Tuned_F1': metrics_cb['F1_Macro'], 'Tuned_LogLoss': metrics_cb['Log_Loss']},
    {'Model': 'LightGBM', 'Tuned_F1': metrics_lgbm['F1_Macro'], 'Tuned_LogLoss': metrics_lgbm['Log_Loss']},
    {'Model': 'XGBoost', 'Tuned_F1': metrics_xgb['F1_Macro'], 'Tuned_LogLoss': metrics_xgb['Log_Loss']}
]
df_tuned = pd.DataFrame(tuned_results)

# Merge
df_final = pd.merge(df_base, df_tuned, on="Model", how="inner")

# Calculate Lift (Improvement in LogLoss)
# Note: Lower LogLoss is better, so (Default - Tuned) is positive improvement
df_final['LogLoss_Lift_Pct'] = ((df_final['Default_LogLoss'] - df_final['Tuned_LogLoss']) / df_final['Default_LogLoss']) * 100
df_final['F1_Lift_Pct'] = ((df_final['Tuned_F1'] - df_final['Default_F1']) / df_final['Default_F1']) * 100

# Sort by Tuned LogLoss (Best Confidence)
df_final = df_final.sort_values("Tuned_LogLoss", ascending=True)

# Save Final Leaderboard
df_final.to_csv("data/final_tuning_leaderboard.csv", index=False)
print("\n[SAVED] Final comparison saved to: data/final_tuning_leaderboard.csv")

# Display
print("\n=== Tuning Impact Analysis ===")
print(df_final.to_markdown(index=False, floatfmt=".5f"))

# 4. Export the Champion Model (The one with lowest Tuned LogLoss)
champion_model_name = df_final.iloc[0]['Model']
print(f"\n🏆 The Final Champion Model is: {champion_model_name}")

if champion_model_name == 'CatBoost':
    champion_model = final_cb
elif champion_model_name == 'LightGBM':
    champion_model = final_lgbm
else:
    champion_model = final_xgb

# Save the actual model object for Task 6
import joblib
joblib.dump(champion_model, "data/champion_model.pkl")
print(f"[SAVED] Champion model serialized to data/champion_model.pkl")


=== Final Evaluation: Default vs. Tuned ===
[Tuned CatBoost Performance]
    F1-Score (Macro):    0.9984  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9974
    Log Loss:            0.0088
[Tuned LightGBM Performance]
    F1-Score (Macro):    0.9980  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9966
    Log Loss:            0.0080
[Tuned XGBoost Performance]
    F1-Score (Macro):    0.9981  <-- EQUAL WEIGHT (Fairness)
    F1-Score (Weighted): 0.9966
    Log Loss:            0.0084

[SAVED] Final comparison saved to: data/final_tuning_leaderboard.csv

=== Tuning Impact Analysis ===
| Model    |   Default_F1 |   Default_LogLoss |   Tuned_F1 |   Tuned_LogLoss |   LogLoss_Lift_Pct |   F1_Lift_Pct |
|:---------|-------------:|------------------:|-----------:|----------------:|-------------------:|--------------:|
| LightGBM |      0.99803 |           0.00968 |    0.99801 |         0.00803 |           17.07618 |      -0.00113 |
| XGBoost  |      0.99789 |           0.0092

### **II.3.1. Optimization Results**

We subjected our top 3 candidates to rigorous Bayesian Optimization using Optuna (30 trials each). The goal was to minimize **Log Loss**, forcing the models to be not just "correct" but "confident."

| Model | Default Log Loss | Tuned Log Loss | **Confidence Lift** | Tuned Macro F1 |
| :--- | :--- | :--- | :--- | :--- |
| **LightGBM** | 0.0097 | **0.0080** | **+17.1%** | 0.9980 |
| **XGBoost** | 0.0092 | 0.0084 | +8.6% | 0.9981 |
| **CatBoost** | 0.0134 | 0.0088 | **+34.6%** | 0.9984 |

### **II.3.2. Key Findings**

**A. The "Confidence" Boost**

While the Classification Accuracy (F1-Score) remained static at ~99.8% (because the clusters are geometrically distinct), the **Log Loss** saw massive improvements.
*   **CatBoost** saw a stunning **34.6% improvement** in probability calibration. The default model was accurate but hesitant; the tuned model is precise and decisive.
*   **LightGBM** emerged as the statistical leader with the absolute lowest Log Loss (**0.0080**), edging out XGBoost and CatBoost by a razor-thin margin.

**B. The Winning Architecture: LightGBM**

LightGBM is selected as the **Champion Model** for the Interpretability Phase (Task 6).

*   **Why?** It achieves the "Holy Trinity" of machine learning:
    1.  **Highest Confidence:** Lowest Log Loss (0.0080).
    2.  **Perfect Stability:** Zero degradation in F1-Score during regularization.
    3.  **Efficiency:** It trains significantly faster than CatBoost or XGBoost, making it ideal for the computationally expensive SHAP calculations coming next.

**C. Strategic Implication**

We have successfully transitioned from a "Black Box" K-Means algorithm to a **Supervised Surrogate (LightGBM)** that mimics the clustering logic with 99.8% fidelity. This authorizes us to use **SHAP (SHapley Additive exPlanations)** on the LightGBM model to explain exactly *why* a customer falls into Cluster 0, 1, 2, or 3.

$\Rightarrow$ **Next Step:** We proceed to **Task: Interpretability & Feature Importance**, where we will ask the LightGBM model to reveal the business drivers behind the segments.

# **III. Visualization (Presentation)**

In [16]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os

# Configuration
FIGURES_DIR = "figures/"
os.makedirs(FIGURES_DIR, exist_ok=True)

print("\n=== III. Visualizing Results ===")

# --- LOAD DATA ---
try:
    df_all = pd.read_csv("data/phase_a_leaderboard.csv")
    df_tuned = pd.read_csv("data/final_tuning_leaderboard.csv")
except FileNotFoundError:
    print("[ERROR] Leaderboard files not found. Run Section I and II first.")



=== III. Visualizing Results ===


In [25]:
# --- FIGURE 1: The Accuracy vs. Confidence Landscape (Sidebar Layout) ---

# 1. Create Categories
def categorize_model(name):
    if name in ['CatBoost', 'XGBoost', 'LightGBM']: return 'Gradient Boosting'
    if name in ['Random Forest', 'Extra Trees']: return 'Ensemble (Bagging)'
    return 'Baseline (Linear/KNN/NN)'

df_all['Category'] = df_all['Model'].apply(categorize_model)

# 2. Plot with Log X Axis
fig1 = px.scatter(
    df_all,
    x="Val_LogLoss",
    y="Val_F1_Macro",
    color="Category",
    symbol="Category",
    size=[60]*len(df_all), # Large, clear dots
    hover_data=["Model", "Overfitting_Gap"],
    title="<b>Model Landscape: Accuracy vs. Confidence</b><br><sup>Log-Scale X-axis reveals the massive gap between Gradient Boosting and Baselines.</sup>",
    color_discrete_map={
        'Gradient Boosting': '#2ca02c', # Green
        'Ensemble (Bagging)': '#1f77b4', # Blue
        'Baseline (Linear/KNN/NN)': '#7f7f7f' # Grey
    },
    log_x=True 
)

# 3. Layout Adjustments
fig1.update_layout(
    height=850, 
    width=1600, # Wide enough to hold the sidebar
    xaxis_title="<b>Validation Log Loss (Lower is Better) - Log Scale</b>",
    yaxis_title="<b>Validation Macro F1 (Higher is Better)</b>",
    template="plotly_white",
    
    # Axis Control: Best Models (Low Loss) on the LEFT
    xaxis=dict(autorange="reversed"), 
    
    # Font Sizes
    font=dict(size=16),
    title_font_size=26,
    
    # --- SIDEBAR CONFIGURATION ---
    # Reserve massive space on the right (400px)
    margin=dict(l=130, r=350, b=120),
    
    # Legend Position (Right Sidebar, Middle-Top)
    legend=dict(
        yanchor="middle", y=0.6,  # Vertically centered slightly up
        xanchor="left", x=1.05,   # Push into the Right Margin
        bgcolor="rgba(255,255,255,0)",
        bordercolor="#333",
        borderwidth=1,
        title_font_family="Arial",
        font=dict(size=14)
    )
)

# 4. Add "Champion Zone" Rectangle (On the Plot)
fig1.add_shape(
    type="rect",
    xref="x", yref="y",
    x0=0.0075, x1=0.018, 
    y0=0.9975, y1=0.9990,
    line=dict(color="#d62728", width=3, dash="dot"), # Red dotted line
    fillcolor="rgba(214, 39, 40, 0.1)", # Faint red fill
)

# 5. Add Annotation (Right Sidebar, Middle-Bottom)
fig1.add_annotation(
    xref="paper", yref="paper",
    x=1.26, # Align with Legend in Right Margin
    y=0.2,  # Vertically centered slightly down
    text="<b>The Champion Zone</b><br><i>(CatBoost, LightGBM, XGBoost)</i><br><br>These models achieve<br><b>< 0.01 Log Loss</b><br>while maintaining<br><b>> 99.8% Accuracy</b>.",
    showarrow=False, # We use a separate arrow line below
    align="left",
    font=dict(size=15, color="#d62728"),
    bordercolor="#d62728",
    borderwidth=2,
    borderpad=15,
    bgcolor="white"
)

# 6. Draw Arrow from Sidebar to Plot Box
# Since annotation is in "paper" coords and box is in "x/y" coords, 
# we use a helper annotation just for the arrow.
fig1.add_annotation(
    xref="paper", yref="y",
    x=1.05, # Start at the sidebar
    y=0.9982, # Point to the cluster's Y height
    axref="x", ayref="y",
    ax=0.018, # End at the right edge of the Red Box
    ay=0.9982,
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#d62728"
)

# Save
fig1.write_image(f"{FIGURES_DIR}task5_fig1_model_landscape.png", scale=3, width=1600, height=850)
fig1.show()
print(f"[OUTPUT] Figure 1 saved to {FIGURES_DIR}")

[OUTPUT] Figure 1 saved to figures/


In [20]:
# --- FIGURE 2: The Tuning Lift (Before vs. After) ---
# Goal: Visualize the massive drop in Log Loss for the Top 3 models.

# 1. Reshape Data for Grouped Bar Chart
df_viz = df_tuned[['Model', 'Default_LogLoss', 'Tuned_LogLoss']].melt(
    id_vars='Model', 
    var_name='Version', 
    value_name='LogLoss'
)
df_viz['Version'] = df_viz['Version'].replace({'Default_LogLoss': 'Default', 'Tuned_LogLoss': 'Tuned'})

# 2. Plot
fig2 = px.bar(
    df_viz,
    x="Model",
    y="LogLoss",
    color="Version",
    barmode="group",
    text_auto=".4f",
    title="<b>Hyperparameter Tuning Impact: Confidence Gain</b><br><sup>CatBoost and LightGBM saw massive reductions in error (Log Loss).</sup>",
    color_discrete_map={'Default': '#aec7e8', 'Tuned': '#1f77b4'} # Light Blue vs Dark Blue
)

fig2.update_layout(
    height=700,
    width=1400,
    margin=dict(l=120, r=150),
    yaxis_title="<b>Log Loss (Lower is Better)</b>",
    template="plotly_white",
    font=dict(size=14),
    legend_title_text=""
)

# Save
fig2.write_image(f"{FIGURES_DIR}task5_fig2_tuning_lift.png", scale=3, width=1400, height=700)
fig2.show()
print(f"[OUTPUT] Figure 2 saved to {FIGURES_DIR}")

[OUTPUT] Figure 2 saved to figures/
